This script is used to extract values for VIIRS surface reflectance (NOAA/VIIRS/001/VNP09GA) from site locations.

It requires a site location file, "new_site_data_o.csv"

Values are exported as .csv files for each site, after being extracted at point locations. Although we provide options to extract values for other collections (Sentinel-1, 2 etc.), we only use this script for VIIRS surface reflectance, as other collections require different processing.

Files are saved to Google Drive in a folder "batch_export_sept_23" (or rename).

In [ ]:
!pip install geemap


In [ ]:
import ee
import geemap
import os
import pandas as pd

In [ ]:
Map = geemap.Map()
Map

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


Load site data

In [ ]:
#table = ee.FeatureCollection("projects/ee-michaeljcecil-phd/assets/all_pod_locations")
site_data = pd.read_csv("/content/gdrive/MyDrive/gee_phd/site_locations/new_site_data_o.csv")
#site_data_day = pd.read_csv("/content/gdrive/MyDrive/gee_phd/site_locations/all_sites_by_day.csv")
site_data
#site_data_day

In [ ]:
len(site_data)
#len(site_data_day)

611

In [ ]:
def extract_pt_coll(coll, pts, start_date, end_date, scale, buffer, site_name, coll_name, out_folder, out_cols):
 # pt_buffer = ee.Feature(pt).buffer(buffer)
  def process_batch(image):
        image = image.unmask()
        date = image.get('system:time_start')
        index = image.get('system:index')
        ft = image.reduceRegions(pts, ee.Reducer.mean(), scale)
        ft = ft.map(lambda f: f.set('date', date))
        ft = ft.map(lambda f: f.set(site_name + '_index', index))
        return ft


  s2 = (
    ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
      .filterDate(start_date, end_date)
     # .select(band_selection)
      .filterBounds(ee.FeatureCollection(pts.geometry()))
  )
   # s2x = s2.filterBounds(ee.FeatureCollection(pts.geometry()))
   # current_pnt = n.get(i).getInfo()



  batch_vals = s2.map(process_batch).flatten()
 #   batch_vals = batch_vals.filter(ee.Filter.notNull(["B2"]))

    # # batch_vals = (
    # #     ee.FeatureCollection(batch_vals)
    # #       .filter(ee.Filter.notNull(["B2"]))
    # # )
    # # var batch_vals =  s2_4extract.map(process_batch).flatten()

  # exp_task = ee.batch.Export.table.toDrive(
  #       collection=batch_vals,
  #       # description='S2_' + region + '_' + str(current_pnt) +"tt",
  #       description= coll_name + '_' + site_name,
  #       folder= out_folder,
  #       selectors=(out_cols)
  #   )
  # exp_task.start()
  print(coll_name + '_' + site_name + ' started')
  return(batch_vals)




In [ ]:
fc = ee.FeatureCollection("projects/zari-331802/assets/new_site_data_o")
print(fc.size().getInfo())

611


Loop through sites and extract values.

In [ ]:
import datetime
#output_path = "/content/gdrive/MyDrive/gee_phd/gee_coll_export"
coll_list = [["COPERNICUS/S2_SR_HARMONIZED", "s2_h", 10],
             ["COPERNICUS/S1_GRD", "s1D_50m_", 10],
             ["ECMWF/ERA5_LAND/DAILY_RAW", "ERA5", 11000],
             ["UCSB-CHG/CHIRPS/DAILY", "CHIRPS", 5000],
             ["NOAA/VIIRS/001/VNP21A1D", "VIIRS_dT", 1000],
             ["NOAA/VIIRS/001/VNP21A1N", "VIIRS_nT", 1000],
             ["NOAA/VIIRS/001/VNP09GA", "VIIRS_sr", 500],
             ["MODIS/061/MOD09GA", "MOD09GA", 500],
             ["MODIS/061/MYD09GA", "MYD09GA", 500],
             ["NOAA/VIIRS/001/VNP22Q2",'VIIRS_LSP' , 500],
             ['MODIS/061/MCD12Q2','MODIS_LSP' , 500]]

#modis = ee.ImageCollection("MODIS/061/MYD09GA")
#output_path = "/content/gdrive/MyDrive/gee_phd/"
#coll_list = ["COPERNICUS/S2_SR_HARMONIZED", "MODIS/061/MOD09GA", "MODIS/061/MYD09GA", "MODIS/006/MCD19A2_GRANULES"]
#coll_name_list = ["s2_h", "mod096a", "myd096a", "mcd19a2"]
#scale_list = [10, 500, 500, 1000]
#start_date = "2020-11-01"
#end_date = "2021-05-31"

#start_date = "2020-11-01"
#end_date = "2021-05-31"
CLOUD_FILTER = 80
CLD_PRB_THRESH = 50
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST = 1
BUFFER = 50
BUFFER_FLAG = False

include_image_props = False
pts = ee.FeatureCollection("projects/ee-michaeljcecil-phd/assets/all_pod_locations")

for k in range(6, 7): ##  loop through collection list
  coll_data = coll_list[k]
  print(coll_data)
  coll = coll_data[0]
  coll_name = coll_data[1]
#  scale = coll_data[2]
  scale = 10


  for j in range(len(site_data)):
    print(j)
    start_date = site_data.at[j, "new_start"]
   # start_date = datetime.datetime.strptime(start_date, '%Y-%m-%d').strftime('%Y-%m-%dT00:00:00.000Z')
 # site_data.at[k, "Location"] = start_date
    end_date = site_data.at[j, "new_end"]
 #   end_date = datetime.datetime.strptime(end_date, '%Y-%m-%d') #+ datetime.timedelta(days=1)
  #  end_date = end_date.strftime('%Y-%m-%dT00:00:00.000Z')
    print(site_data.at[j, "Location"])
    if coll in ["NOAA/VIIRS/001/VNP22Q2", 'MODIS/061/MCD12Q2']:
      start_date = start_date[0:4] + "-01-01"
    print(start_date, " ", end_date)

    site_long = site_data.at[j, "new_start"]
    site_lat = site_data.at[j, "new_start"]

  #  stats = coll_export.reduceRegion()

    current_location = site_data.at[j, "Location"]

    current_pt = ee.Feature(fc.filter(ee.Filter.eq("Location", current_location)).first()) #.filter(ee.Filter.gt("date", "2018-12-14"))

    if BUFFER_FLAG:
      current_pt = current_pt.geometry().buffer(BUFFER)
    else:
      current_pt = current_pt.geometry()
   # print(current_pt)
  #  print(current_pt.getInfo())

    ## s1 filters
    if coll_data == "COPERNICUS/S1_GRD":
      coll_export = ee.ImageCollection(coll).filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING')).filterDate(start_date, end_date).filterBounds(ee.FeatureCollection(current_pt).geometry()).toBands()
    else:
      coll_export = ee.ImageCollection(coll).filterDate(start_date, end_date).filterBounds(ee.FeatureCollection(current_pt).geometry()).toBands()
    if coll_data == "NOAA/VIIRS/001/VNP09GA":
      coll_export = coll_export.select([''])


    data = coll_export.reduceRegion(ee.Reducer.median(), current_pt, scale)
    exp_task = ee.batch.Export.table.toDrive(
          collection= ee.FeatureCollection([ee.Feature(None, data)]),
          # description='S2_' + region + '_' + str(current_pnt) +"tt",
          description= coll_name + "_" + current_location + "_" + str(start_date) + "_" + str(end_date) + '_export',
          folder= 'batch_export_sept_23'
          #selectors=(out_cols)
    )
    exp_task.start()


